In [1]:
import gzip
from correctionlib.schemav2 import CorrectionSet,Correction,Category,CategoryItem, Formula, MultiBinning
from itertools import chain

In [2]:
def getdict(file):
    import yaml
    # opening a file
    with open(file, 'r') as stream:
        try:
            d=yaml.safe_load(stream)
        except yaml.YAMLError as e:
            print(e)
    #k={"2018":{"HLT_Photon120": {315257: {1: 49}, 315259: {1: 49}, 315264: {32: 74}, 315265: {4: 74}}}}
    return d

In [3]:
def dictf(prescale_dict,year,HLT,run,lumi):
    try:
        dd=prescale_dict[year][HLT][run][lumi]
    except:
        dd=-1
    return dd
 

In [4]:
import copy
def getmultibinning(triggerlumilist):
    triggerlumilist1=copy.deepcopy(triggerlumilist)#.deepcopy()
    if triggerlumilist[0]!=1:
        triggerlumilist1.insert(0, 1)
    tot=len(triggerlumilist1)
    triggerlumilist1.insert(tot, 99999)
    return triggerlumilist1

def prescaletrick(prescalelist,lumilist_):
    if lumilist_[0]!=1:
        prescalelist.insert(0, 1)
    return prescalelist

def getvalue(prescale_dict,year,HLT,run):
    print(year,HLT,run)
    lumilist_= list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT][run]])))
    print(lumilist_)
    print([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_])
    lumilist_.sort()
    print(lumilist_)
    #print(getmultibinning(lumilist_))
    #print(lumilist_)
    print([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_])
    #print([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_])
    #print(prescaletrick([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_],lumilist_))
    value=MultiBinning.parse_obj({
        "inputs":["lumi"],
        "nodetype": "multibinning",
        "edges": [
            getmultibinning(lumilist_)
        ],
        "content": prescaletrick([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_],lumilist_)  ,
        "flow": 'error',
    })
    print(prescaletrick([dictf(prescale_dict,year,HLT,run,lumi) for lumi in lumilist_],lumilist_))
    return value

def getdictpre(years):
    prescale_dict={'2018':getdict("2018_UL/photon_prescales_2018.yaml.txt"),
                   '2016':getdict("2016_UL/photon_prescales_2016.yaml"),
                   '2017':getdict("2017_UL/photon_prescales_2017.yaml")}
    
    HLTs = list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year] for year in years])))
    #for year in years:
    #    runs = list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT] for HLT in HLTs])))
    #for year in years:
    #    for HLT in HLTs:
    #        lumis = list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT][run] for run in runs])))
    output = Category.parse_obj({
                "nodetype": "category",
                "input": "HLT",
                "content":[
                    CategoryItem.parse_obj({
                        "key": HLT, 
                        "value":Category.parse_obj({
                            "nodetype": "category",
                            "input": "run",
                            "content":[
                                CategoryItem.parse_obj({
                                    "key": run, 
                                    "value": getvalue(prescale_dict,year,HLT,run)
                                })
                                for run in list(set(chain.from_iterable(value.keys() for value in [prescale_dict[year][HLT]])))
                            ],
                        })
                    })
                    for HLT in HLTs
                ],
    })
    return output


In [5]:
years=["2018","2017","2016"]

for year in years:
    corrs=[]
    corr = Correction.parse_obj(
        {
            "version": 2,
            "name": "HLT_prescale",
            "description": f"Testing prescale",
            "inputs": [
                {"name": "year","type": "string", "description": "year/scenario: example 2016preVFP, 2017 etc"},
                {"name": "HLT","type": "string", "description": "HLT name"},
                {"name": "run","type": "int", "description": "run "},
                {"name": "lumi","type": "real"},
            ],
            "output": {"name": "prescale", "type": "real", "description": "value of prescale"},
            "data": Category.parse_obj({
                "nodetype": "category",
                "input": "year",
                "content": [
                CategoryItem.parse_obj({"key":year,
                                        "value": getdictpre([year])})]
            })
        })

    corrs.append(corr)


    #Save JSON
    cset = CorrectionSet(schema_version=2, corrections=corrs,description=f"Trigger prescales- preliminary")
    with open(f"prescale_{year}.json", "w") as fout:
        fout.write(cset.json(exclude_unset=True, indent=4))

2018 HLT_Photon300_NoHE 319503
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 315420
[28]
[1]
[28]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 317475
[33]
[1]
[33]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 319524
[36]
[1]
[36]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 317478
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 319526
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 319528
[6]
[1]
[6]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 317484
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 317488
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 317527
[41]
[1]
[41]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 319579
[41]
[1]
[41]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 315488
[42]
[1]
[42]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 315489
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 315490
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 323693
[38]
[1]
[38]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 323696
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 315506
[13]
[1]
[13]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 315510
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 323702
[1]
[1]


[1]
[1]
[1]
2018 HLT_Photon300_NoHE 324318
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 316153
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 322317
[48]
[1]
[48]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 322319
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 322322
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 322324
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 316186
[38]
[1]
[38]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 316187
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 322332
[37]
[1]
[37]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 316199
[33]
[1]
[33]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 316200
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 316201
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 316202
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 322348
[40]
[1]
[40]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 322355
[36]
[1]
[36]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 322356
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 316216
[25]
[1]
[25]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 316217
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 316218
[1]


[1]
[1]
[1]
2018 HLT_Photon300_NoHE 315270
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 317319
[44]
[1]
[44]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 317320
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 321414
[31]
[1]
[31]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 321415
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 323470
[38]
[1]
[38]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 323471
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 323472
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 323473
[3]
[1]
[3]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 323474
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 323475
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 321431
[30]
[1]
[30]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 321432
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 321433
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 317338
[66]
[1]
[66]
[1]
[1, 1]
2018 HLT_Photon300_NoHE 317339
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 317340
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 321434
[1]
[1]
[1]
[1]
[1]
2018 HLT_Photon300_NoHE 321436
[1]
[1]
[1]

2018 HLT_Photon75_R9Id90_HE10_IsoM 323997
[3]
[64]
[3]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 319908
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 319909
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 319910
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 319912
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 319913
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 319914
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 319915
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 321960
[18]
[64]
[18]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 321961
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 321973
[37]
[64]
[37]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324021
[44]
[64]
[44]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 321975
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324022
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 315840
[33]
[62]
[33]
[62]
[1, 62]
20

2018 HLT_Photon75_R9Id90_HE10_IsoM 320674
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324769
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324772
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 316590
[17]
[64]
[17]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 320688
[49]
[64]
[49]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324785
[77]
[64]
[77]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324791
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 316613
[49]
[64]
[49]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 316615
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 320712
[39]
[64]
[39]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324835
[40]
[64]
[40]
[64]
[1, 64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324840
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324841
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 324846
[1]
[64]
[1]
[64]
[64]
2018 HLT_Photon75_R9Id90_HE10_IsoM 320757
[51]
[64]
[51]
[64

...

[1, 9, 504]
[500, 420, 360]
[500, 420, 360]
2016 HLT_Photon30_R9Id90_HE10_IsoM 282034
[1]
[700]
[1]
[700]
[700]
2016 HLT_Photon30_R9Id90_HE10_IsoM 282035
[1, 34, 12]
[700, 630, 600]
[1, 12, 34]
[700, 600, 630]
[700, 600, 630]
2016 HLT_Photon30_R9Id90_HE10_IsoM 282037
[1, 865, 67, 645, 426, 203, 15, 1490]
[700, 450, 700, 510, 630, 600, 600, 300]
[1, 15, 67, 203, 426, 645, 865, 1490]
[700, 600, 700, 600, 630, 510, 450, 300]
[700, 600, 700, 600, 630, 510, 450, 300]
2016 HLT_Photon30_R9Id90_HE10_IsoM 279993
[89, 85]
[800, 700]
[85, 89]
[700, 800]
[1, 700, 800]
2016 HLT_Photon30_R9Id90_HE10_IsoM 279994
[1]
[800]
[1]
[800]
[800]
2016 HLT_Photon30_R9Id90_HE10_IsoM 275911
[259, 62]
[420, 500]
[62, 259]
[500, 420]
[1, 500, 420]
2016 HLT_Photon30_R9Id90_HE10_IsoM 275912
[1, 261]
[420, 360]
[1, 261]
[420, 360]
[420, 360]
2016 HLT_Photon30_R9Id90_HE10_IsoM 275913
[1]
[360]
[1]
[360]
[360]
2016 HLT_Photon30_R9Id90_HE10_IsoM 280013
[1]
[700]
[1]
[700]
[700]
2016 HLT_Photon30_R9Id90_HE10_IsoM 275918


[240, 180]
[240, 180]
2016 HLT_Photon30_R9Id90_HE10_IsoM 278509
[100, 529, 691, 1044, 91, 191]
[700, 630, 510, 450, 600, 600]
[91, 100, 191, 529, 691, 1044]
[600, 700, 600, 630, 510, 450]
[1, 600, 700, 600, 630, 510, 450]
2016 HLT_Photon30_R9Id90_HE10_IsoM 274420
[94]
[360]
[94]
[360]
[1, 360]
2016 HLT_Photon30_R9Id90_HE10_IsoM 274421
[1]
[360]
[1]
[360]
[360]
2016 HLT_Photon30_R9Id90_HE10_IsoM 274422
[1, 363, 1734, 1159]
[360, 240, 100, 180]
[1, 363, 1159, 1734]
[360, 240, 180, 100]
[360, 240, 180, 100]
2016 HLT_Photon250_NoHE 274440
[92]
[1]
[92]
[1]
[1, 1]
2016 HLT_Photon250_NoHE 274441
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 274442
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 276495
[87]
[1]
[87]
[1]
[1, 1]
2016 HLT_Photon250_NoHE 276501
[4]
[1]
[4]
[1]
[1, 1]
2016 HLT_Photon250_NoHE 276502
[2]
[1]
[2]
[1]
[1, 1]
2016 HLT_Photon250_NoHE 276525
[88]
[1]
[88]
[1]
[1, 1]
2016 HLT_Photon250_NoHE 276527
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 276528
[4]
[1]
[4]
[1]
[1, 1]
2016 HL

[1]
[1, 1]
2016 HLT_Photon250_NoHE 284014
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 281975
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 275832
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 275833
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 275834
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 275835
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 275836
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 275837
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 279931
[84]
[1]
[84]
[1]
[1, 1]
2016 HLT_Photon250_NoHE 281976
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 284025
[110]
[1]
[110]
[1]
[1, 1]
2016 HLT_Photon250_NoHE 284029
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 284035
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 284036
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 284037
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 284038
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 275847
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 284039
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon250_NoHE 284040
[1]
[1]
[1]
[1]
[1]
2016 HLT

[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279667
[68]
[1]
[68]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 281726
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 281727
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279681
[77]
[1]
[77]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279682
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279683
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279684
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279685
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279691
[71]
[1]
[71]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279694
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 273554
[77]
[1]
[77]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 273555
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279715
[71]
[1]
[71]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 279716
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 283820
[67]
[1]
[67]
[1]
[1, 1]
2016 HLT_Photo

[3]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 280383
[64]
[1]
[64]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 274240
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 274241
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 280384
[2]
[1]
[2]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 280385
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 274244
[15]
[1]
[15]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 278345
[84]
[1]
[84]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 274250
[2]
[1]
[2]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 274251
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 278346
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 278349
[1]
[1]
[1]
[1]
[1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 276315
[40]
[1]
[40]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 276317
[3]
[1]
[3]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 276318
[3]
[1]
[3]
[1]
[1, 1]
2016 HLT_Photon165_R9Id90_HE10_IsoM 278366
[1]
[1]
[1]
[1]
[1]
2016 HLT

In [6]:
from correctionlib import _core

evaluator = _core.CorrectionSet.from_file('prescale_2018.json')

In [7]:
evaluator["HLT_prescale"].evaluate("2018","HLT_Photon165_R9Id90_HE10_IsoM",321887,99.0)

4.0

In [8]:
evaluator["HLT_prescale"].evaluate("2018","HLT_Photon165_R9Id90_HE10_IsoM",321887,99.0)

4.0